In [2]:
import pandas as pd
from sklearn.neighbors import KernelDensity
from sklearn.metrics import precision_score, recall_score, f1_score

In [19]:
data = pd.read_csv("C:/Users/seohe/OneDrive/바탕 화면/Thesis/dataset_cluster.csv")

In [25]:
# 라벨링1인 데이터셋
cluster1 = pd.read_csv("C:/Users/seohe/OneDrive/바탕 화면/Thesis/dataset_cluster_1.csv")

In [26]:
# 좌표 데이터와 Venue ID 추출
coord = cluster1[['Latitude', 'Longitude']]
venue_ids = cluster1['Venue ID'].tolist()

In [27]:
# KDE 학습
kde = KernelDensity(bandwidth=0.01, metric='haversine', kernel='gaussian')
kde.fit(coord)

KernelDensity(bandwidth=0.01, metric='haversine')

In [28]:
# 추천 POI 리스트 생성
target_user_coord = [[40.7589,-73.9851]] # 샘플 좌표, 실제 데이터에는 없는 좌표
poi_scores = kde.score_samples(coord)
k = len(venue_ids) # 현재 클러스터에서의 POI 개수로 추천되는 POI 개수를 설정함
recommended_poi_indices = poi_scores.argsort()[::-1][:k]
recommended_poi_ids = [venue_ids[i] for i in recommended_poi_indices]

In [29]:
# 정밀도, 재현율, F1 평균 계산
avg_precision = 0
avg_recall = 0
avg_f1 = 0
num_clusters = cluster1['Cluster'].nunique()

In [31]:
print(cluster1)

       Unnamed: 0  User ID  Location Frequency  Activity  Main Activity Time  \
0               0        1                 0.0         0                  22   
1               1        1                 0.0         0                  22   
2               2        1                 0.0         0                  22   
3               3        1                 0.0         0                  22   
4               4        1                 0.0         0                  22   
...           ...      ...                 ...       ...                 ...   
65114       65114     1083                 0.0       961                   0   
65115       65115     1083                 0.0       961                   0   
65116       65116     1083                 0.0       961                   0   
65117       65117     1083                 0.0       961                   0   
65118       65118     1083                 0.0       961                   0   

       Cluster  Unnamed: 0.1           

In [34]:
for cluster_id in range(1, num_clusters + 1):
    # 라벨링1인 클러스터 추출
    cluster = cluster1[cluster1['Cluster']==cluster_id]
    
    # 좌표 데이터와 Venue ID 추출
    coord = cluster1[['Latitude', 'Longitude']]
    venue_ids = cluster1['Venue ID'].tolist()

    # KDE 학습
    kde = KernelDensity(bandwidth=0.01, metric='haversine', kernel='gaussian')
    kde.fit(coord)

    # 추천 POI 리스트 생성
    target_user_coord = [[cluster['Latitude'].iloc[0], cluster['Longitude'].iloc[0]]]
    poi_scores = kde.score_samples(coord)
    k = len(venue_ids) # 현재 클러스터에서의 POI 개수로 추천되는 POI 개수를 설정함
    recommended_poi_indices = poi_scores.argsort()[::-1][:k]
    recommended_poi_ids = [venue_ids[i] for i in recommended_poi_indices]

    # 정답 POI ID 추출
    target_poi_ids = cluster1[cluster1['Cluster']==cluster_id]['Venue ID'].tolist()

    # 정밀도, 재현율, F1 계산
    precision = precision_score(target_poi_ids, recommended_poi_ids)
    recall = recall_score(target_poi_ids, recommended_poi_ids)
    f1 = f1_score(target_poi_ids, recommended_poi_ids)

    avg_precision += precision
    avg_recall += recall
    avg_f1 += f1

ValueError: Found input variables with inconsistent numbers of samples: [3009, 65119]

In [ ]:
print(f"Average Precision: {avg_precision:.4f}")
print(f"Average Recall: {avg_recall:.4f}")
print(f"Average F1: {avg_f1:.4f}")